In [1]:
import os
import copy
import numpy as np
import pandas as pd
import torch

from pruning import NetworkOptimization

def save_csv(arrs, pth, epoch):
    df = pd.DataFrame(np.array(arrs), columns=range(1, epoch + 1))
    if os.path.exists(pth):
        arrs_old = pd.read_csv(pth, index_col=0).values.reshape(-1, epoch)
        arrs = np.concatenate((arrs_old, np.array(arrs)), axis=0)
        df = pd.DataFrame(arrs, columns=range(1, epoch + 1))

    df.to_csv(pth)

In [2]:
epoch = 30  ## Epoch size
batch = 150 ## Batch size.
lr = 0.001 ## Learning rate.
data = 'mnist' ## Select data mnist or fashion.
edge = 5 ## Number of edge changes.
eta = 0.98 ## Decay coef.
metro = '1,10,50' 
reduce = '10,20,30,40,50,60,70,80,90,95,99'

REPEAT = 5
METRO = np.array(metro.split(',')).astype(int)
REDUCE = np.array(reduce.split(',')).astype(float) * 0.01

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('\n[v] Using device: {}'.format(device))

layer = ['fc1','fc2']


[v] Using device: cpu


In [3]:
nnOpt = copy.deepcopy(NetworkOptimization(batch_size=batch,
                                        learning_rate=lr,
                                        mask_ratio=[
                                        0, 0.0, 0.0, 0],
                                        eta=eta, data=data,
                                        edge_changed=edge))
losses, accs, epoch = nnOpt.fit(epoch_num=10,  # EPOCH
                                save_step=np.inf)
        

0.9304och: 1 | test acc: 0.93 | test loss: 0.442
0.9458och: 2 | test acc: 0.946 | test loss: 0.305
0.9489och: 3 | test acc: 0.949 | test loss: 0.246
0.9537och: 4 | test acc: 0.954 | test loss: 0.21
0.9559och: 5 | test acc: 0.956 | test loss: 0.186
0.9574och: 6 | test acc: 0.957 | test loss: 0.171
0.9582och: 7 | test acc: 0.958 | test loss: 0.16
0.958poch: 8 | test acc: 0.958 | test loss: 0.154
0.9603och: 9 | test acc: 0.96 | test loss: 0.147
0.9608och: 10 | test acc: 0.961 | test loss: 0.142


# SA

In [11]:
print('[v] Start SA method.')
folder = '{}_edge{}_SA/performance_under_different_M'.format(data, edge)
print('Save to: ', folder)
if not os.path.exists(folder):
    os.makedirs(folder)
                
file_loss_untrained = 'loss_untrained.csv'
file_acc_untrained = 'acc_untrained.csv'
file_loss_trained = 'loss_trained.csv'
file_acc_trained = 'acc_trained.csv'

for time in range(REPEAT):
    print('[v] Start {} time try'.format(time+1))
                
    loss_untrained, acc_untrained = [], []
    loss_trained, acc_trained = [], []
    
    if os.path.exists(os.path.join(folder, file_loss_untrained)):
        arrs_old = pd.read_csv(os.path.join(
            folder, file_loss_untrained), index_col=0).values
        if len(arrs_old) >= REPEAT:
            break    

    loss_untrained.append(losses[-1])
    acc_untrained.append(accs[-1])
    loss_trained.append(losses[-1])
    acc_trained.append(accs[-1])
                
    for r in REDUCE:
        print('[v] Start {:>2}% Reduced Case'.format(r * 100))
            
        nnOpt_SA = copy.deepcopy(nnOpt)
                
        loss, acc, Hq_iter = nnOpt_SA.FCSA(layer, reduce=r, metropolis=1, iteration = 200)
                
        loss_untrained.append(loss)
        acc_untrained.append(acc)

        losses_new, accs_new, epoch = nnOpt_SA.fit(epoch_num=1, save_step=np.inf)
        
        loss_trained.append(losses_new[-1])
        acc_trained.append(accs_new[-1])
                
        del nnOpt_SA

        print('[v] Finish {:>2}% Reduced Case'.format(r * 100))
        # break
        
    save_csv([loss_untrained], os.path.join(folder, file_loss_untrained), epoch=len(loss_untrained))  # EPOCH
    save_csv([acc_untrained], os.path.join(folder, file_acc_untrained), epoch=len(acc_untrained))  # EPOCH
    save_csv([loss_trained], os.path.join(folder, file_loss_trained), epoch=len(loss_trained))  # EPOCH
    save_csv([acc_trained], os.path.join(folder, file_acc_trained), epoch=len(acc_trained))  # EPOCH
    
    print('[v] Finish {} time try'.format(time+1))

print('[v] Finish SA method.')
# break

[v] Start SA method.
Save to:  mnist_edge5_SA/performance_under_different_M
[v] Start 1 time try
[v] Start 10.0% Reduced Case
0.15145818889141083
0.9609och: 1 | test acc: 0.961 | test loss: 0.144t loss: 0.211 | T: 0.003 | M1: 90.0% | M2: 90.0% 
[v] Finish 10.0% Reduced Case
[v] Start 20.0% Reduced Case
0.10833969712257385
0.9587och: 1 | test acc: 0.959 | test loss: 0.15st loss: 0.312 | T: 0.002 | M1: 79.981% | M2: 81.5% 
[v] Finish 20.0% Reduced Case
[v] Start 30.0% Reduced Case
0.6082955002784729
0.9541och: 1 | test acc: 0.954 | test loss: 0.168t loss: 0.547 | T: 0.011 | M1: 70.0% | M2: 70.0% 
[v] Finish 30.0% Reduced Case
[v] Start 40.0% Reduced Case
0.6090712547302246
0.9455och: 1 | test acc: 0.946 | test loss: 0.202t loss: 0.905 | T: 0.011 | M1: 59.987% | M2: 61.0% 
[v] Finish 40.0% Reduced Case
[v] Start 50.0% Reduced Case
0.45531851053237915
0.9462och: 1 | test acc: 0.946 | test loss: 0.199t loss: 0.885 | T: 0.008 | M1: 50.032% | M2: 47.5% 
[v] Finish 50.0% Reduced Case
[v] Start

# SQA

In [12]:
print('[v] Start SQA method.')
for m in METRO:
    folder = '{}_edge{}_SQA/performance_under_different_M/metropolis_{}'.format(data, edge, m)
    print('Save to: ', folder)
    if not os.path.exists(folder):
        os.makedirs(folder)
        
    print('[v] Start {} metro.'.format(m))
        
    file_loss_untrained = 'loss_untrained.csv'
    file_acc_untrained = 'acc_untrained.csv'
    file_loss_trained = 'loss_trained.csv'
    file_acc_trained = 'acc_trained.csv'
    
    for time in range(REPEAT):
        print('[v] Start {} time try under {} M'.format(time + 1, m))
                    
        loss_untrained, acc_untrained = [], []
        loss_trained, acc_trained = [], []

        if os.path.exists(os.path.join(folder, file_loss_untrained)):
            arrs_old = pd.read_csv(os.path.join(
                folder, file_loss_untrained), index_col=0).values
            if len(arrs_old) >= REPEAT:
                break
                        
        loss_untrained.append(losses[-1])
        acc_untrained.append(accs[-1])
        loss_trained.append(losses[-1])
        acc_trained.append(accs[-1])
                
        for r in REDUCE:
            print('[v] Start {:>2}% Reduced Case'.format(r * 100))
            
            nnOpt_SQA = copy.deepcopy(nnOpt)
                
            loss, acc, Hq_iter, data_iter = nnOpt_SQA.FCSQA(layer, reduce=r, metropolis=m, iteration=200)
                
            loss_untrained.append(loss)
            acc_untrained.append(acc)

            losses_new, accs_new, epoch = nnOpt_SQA.fit(epoch_num=1, save_step=np.inf)
            
            loss_trained.append(losses_new[-1])
            acc_trained.append(accs_new[-1])
                
            del nnOpt_SQA

            print('[v] Finish {:>2}% Reduced Case'.format(r * 100))
            # break
        
        save_csv([loss_untrained], os.path.join(folder, file_loss_untrained), epoch=len(loss_untrained))  # EPOCH
        save_csv([acc_untrained], os.path.join(folder, file_acc_untrained), epoch=len(acc_untrained))  # EPOCH
        save_csv([loss_trained], os.path.join(folder, file_loss_trained), epoch=len(loss_trained))  # EPOCH
        save_csv([acc_trained], os.path.join(folder, file_acc_trained), epoch=len(acc_trained))  # EPOCH
        
        print('[v] Finish {} time try under {} M'.format(time + 1, m))
           
    print('[v] Finish {} metro.'.format(m))
    # break
    
print('[v] Finish SQA method.')
# break

[v] Start SQA method.
Save to:  mnist_edge5_SQA/performance_under_different_M/metropolis_1
[v] Start 1 metro.
[v] Start 1 time try under 1 M
[v] Start 10.0% Reduced Case
0.3267217427492142
0.9609och: 1 | test acc: 0.961 | test loss: 0.1439 | M1: 90.0% | M2: 90.0%
[v] Finish 10.0% Reduced Case
[v] Start 20.0% Reduced Case
0.06305387616157532
0.9591och: 1 | test acc: 0.959 | test loss: 0.1513 | M1: 79.994% | M2: 80.5%
[v] Finish 20.0% Reduced Case
[v] Start 30.0% Reduced Case
0.30688726902008057
0.9542och: 1 | test acc: 0.954 | test loss: 0.16433 | M1: 69.955% | M2: 73.5%
[v] Finish 30.0% Reduced Case
[v] Start 40.0% Reduced Case
0.310316264629364
0.9466och: 1 | test acc: 0.947 | test loss: 0.19878 | M1: 60.013% | M2: 59.0%
[v] Finish 40.0% Reduced Case
[v] Start 50.0% Reduced Case
0.4678148627281189
0.9417och: 1 | test acc: 0.942 | test loss: 0.20912 | M1: 49.949% | M2: 54.0%
[v] Finish 50.0% Reduced Case
[v] Start 60.0% Reduced Case
0.8863745927810669
0.9372och: 1 | test acc: 0.937 | t

# SQA based min-K

In [4]:
print('[v] Start SQA based min-K method.')
for m in METRO:
    folder = '{}_edge{}_SQA_based_minK/performance_under_different_M/metropolis_{}'.format(data, edge, m)
    print('Save to: ', folder)
    if not os.path.exists(folder):
        os.makedirs(folder)
        
    print('[v] Start {} metro.'.format(m))
        
    file_loss_untrained = 'loss_untrained.csv'
    file_acc_untrained = 'acc_untrained.csv'
    file_loss_trained = 'loss_trained.csv'
    file_acc_trained = 'acc_trained.csv'
    
    for time in range(REPEAT):
        print('[v] Start {} time try under {} M'.format(time + 1, m))
                    
        loss_untrained, acc_untrained = [], []
        loss_trained, acc_trained = [], []

        if os.path.exists(os.path.join(folder, file_loss_untrained)):
            arrs_old = pd.read_csv(os.path.join(
                folder, file_loss_untrained), index_col=0).values
            if len(arrs_old) >= REPEAT:
                break
                        
        loss_untrained.append(losses[-1])
        acc_untrained.append(accs[-1])
        loss_trained.append(losses[-1])
        acc_trained.append(accs[-1])
                
        for r in REDUCE:
            print('[v] Start {:>2}% Reduced Case'.format(r * 100))
            
            nnOpt_SQA = copy.deepcopy(nnOpt)
                
            loss, acc, Hq_iter, data_iter = nnOpt_SQA.FCSQA_based_minK(layer, reduce=r, metropolis=m, iteration=200)
                
            loss_untrained.append(loss)
            acc_untrained.append(acc)

            losses_new, accs_new, epoch = nnOpt_SQA.fit(epoch_num=1, save_step=np.inf)
            
            loss_trained.append(losses_new[-1])
            acc_trained.append(accs_new[-1])
                
            del nnOpt_SQA

            print('[v] Finish {:>2}% Reduced Case'.format(r * 100))
            # break
        
        save_csv([loss_untrained], os.path.join(folder, file_loss_untrained), epoch=len(loss_untrained))  # EPOCH
        save_csv([acc_untrained], os.path.join(folder, file_acc_untrained), epoch=len(acc_untrained))  # EPOCH
        save_csv([loss_trained], os.path.join(folder, file_loss_trained), epoch=len(loss_trained))  # EPOCH
        save_csv([acc_trained], os.path.join(folder, file_acc_trained), epoch=len(acc_trained))  # EPOCH
        
        print('[v] Finish {} time try under {} M'.format(time + 1, m))
           
    print('[v] Finish {} metro.'.format(m))
    # break
    
print('[v] Finish SQA based min-K method.')
# break

[v] Start SQA based min-K method.
Save to:  mnist_edge5_SQA_based_minK/performance_under_different_M/metropolis_1
[v] Start 1 metro.
[v] Start 1 time try under 1 M
[v] Start 10.0% Reduced Case
0.11449807491898537
0.9585och: 1 | test acc: 0.958 | test loss: 0.14571 | M1: 89.949% | M2: 94.0% 
[v] Finish 10.0% Reduced Case
[v] Start 20.0% Reduced Case
0.1563432890623808
0.9594och: 1 | test acc: 0.959 | test loss: 0.14278 | M1: 79.911% | M2: 87.0% 
[v] Finish 20.0% Reduced Case
[v] Start 30.0% Reduced Case
0.20064258693158626
0.9601och: 1 | test acc: 0.96 | test loss: 0.143208 | M1: 69.77% | M2: 88.0% 
[v] Finish 30.0% Reduced Case
[v] Start 40.0% Reduced Case
0.2638753305673599
0.9591och: 1 | test acc: 0.959 | test loss: 0.14387 | M1: 59.739% | M2: 80.5% 
[v] Finish 40.0% Reduced Case
[v] Start 50.0% Reduced Case
0.42122591587901115
0.9578och: 1 | test acc: 0.958 | test loss: 0.14634 | M1: 49.719% | M2: 72.0% 
[v] Finish 50.0% Reduced Case
[v] Start 60.0% Reduced Case
0.509975064381957
0.

# min-K

In [5]:
print('[v] Start minK method.')
folder = '{}_minK/performance_under_different_M'.format(data)
print('Save to: ', folder)
if not os.path.exists(folder):
    os.makedirs(folder)
        
file_loss_untrained = 'loss_untrained.csv'
file_acc_untrained = 'acc_untrained.csv'
file_loss_trained = 'loss_trained.csv'
file_acc_trained = 'acc_trained.csv'
            
loss_untrained, acc_untrained = [], []
loss_trained, acc_trained = [], []

loss_untrained.append(losses[-1])
acc_untrained.append(accs[-1])
loss_trained.append(losses[-1])
acc_trained.append(accs[-1])
                
for r in REDUCE:
    print('[v] Start {:>2}% Reduced Case'.format(r * 100))
            
    nnOpt_minK = copy.deepcopy(nnOpt)
                
    loss, acc = nnOpt_minK.MINK(layer, reduce=r)
                
    loss_untrained.append(loss)
    acc_untrained.append(acc)

    losses_new, accs_new, epoch = nnOpt_minK.fit(epoch_num=1,  # epoch
                                                            save_step=np.inf)
    loss_trained.append(losses_new[-1])
    acc_trained.append(accs_new[-1])

    del nnOpt_minK

    print('[v] Finish {:>2}% Reduced Case'.format(r * 100))
    # break

        
save_csv([loss_untrained], os.path.join(folder, file_loss_untrained), epoch=len(loss_untrained))  # EPOCH
save_csv([acc_untrained], os.path.join(folder, file_acc_untrained), epoch=len(acc_untrained))  # EPOCH
save_csv([loss_trained], os.path.join(folder, file_loss_trained), epoch=len(loss_trained))  # EPOCH
save_csv([acc_trained], os.path.join(folder, file_acc_trained), epoch=len(acc_trained))  # EPOCH
            
print('[v] Finish minK method.')

[v] Start minK method.
Save to:  mnist_minK/performance_under_different_M
[v] Start 10.0% Reduced Case
0.9758och: 1 | test acc: 0.976 | test loss: 0.09691% | M2: 95.76% | M3: 95.68% | M4: 96.2% 
[v] Finish 10.0% Reduced Case
[v] Start 20.0% Reduced Case
0.9754och: 1 | test acc: 0.975 | test loss: 0.09727% | M2: 91.92% | M3: 91.68% | M4: 93.2% 
[v] Finish 20.0% Reduced Case
[v] Start 30.0% Reduced Case
0.9757och: 1 | test acc: 0.976 | test loss: 0.09305% | M2: 87.72% | M3: 87.0% | M4: 84.2% 
[v] Finish 30.0% Reduced Case
[v] Start 40.0% Reduced Case
0.978poch: 1 | test acc: 0.978 | test loss: 0.08911% | M2: 83.16% | M3: 82.96% | M4: 79.4% 
[v] Finish 40.0% Reduced Case
[v] Start 50.0% Reduced Case


KeyboardInterrupt: 